In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Imports

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV, ElasticNet, BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor, VotingRegressor
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Data

In [2]:
raw_train_data = pd.read_csv("/kaggle/input/playground-series-s3e25/train.csv")
raw_test_data = pd.read_csv("/kaggle/input/playground-series-s3e25/test.csv")

In [3]:
raw_train_data

,id,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness
0,0,100.0,0.841611,10.000000,4.800000,20.612526,11.088100,2.766000,1.732000,0.860000,0.496070,0.91457,6.0
1,1,100.0,7.558488,10.000000,4.800000,20.298893,12.040830,2.755000,1.631000,0.910000,0.492719,0.71760,6.5
2,2,76.0,8.885992,15.600000,5.600000,33.739258,12.086300,2.828000,1.788000,0.864000,0.481478,1.50633,2.5
3,3,100.0,8.795296,10.000000,4.800000,20.213349,10.948500,2.648000,1.626000,0.936000,0.489272,0.78937,6.0
4,4,116.0,9.577996,11.600000,4.800000,24.988133,11.824480,2.766000,1.682000,0.896000,0.492736,1.86481,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10402,10402,128.0,7.558488,12.000000,4.000000,26.385218,11.330440,2.644000,1.631000,0.892000,0.496070,1.79607,4.0
10403,10403,30.0,1.743160,10.000000,5.333333,20.766935,14.163933,3.090000,1.556667,0.866667,0.480390,0.81480,5.0
10404,10404,196.0,30.920000,24.500000,5.500000,53.490297,10.074300,2.295000,1.545000,1.120000,0.469715,2.11540,1.8
10405,10405,38.0,1.553160,12.666667,4.666667,26.621687,11.290033,2.743333,1.756667,0.980000,0.486507,0.77755,6.0


In [4]:
raw_train_data.isna().sum()

id                       0
allelectrons_Total       0
density_Total            0
allelectrons_Average     0
val_e_Average            0
atomicweight_Average     0
ionenergy_Average        0
el_neg_chi_Average       0
R_vdw_element_Average    0
R_cov_element_Average    0
zaratio_Average          0
density_Average          0
Hardness                 0
dtype: int64

wow why is this data so clean lmao

In [5]:
X_train = raw_train_data.drop(columns=["id","Hardness"])
y_train = raw_train_data["Hardness"]
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2)

# Linear Regression

In [6]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_valid)
print("MAE: ", mean_absolute_error(y_valid, y_pred))
print("MSE: ", mean_squared_error(y_valid, y_pred))

MAE:  1.191309031387157
MSE:  2.184903947439542


# Lasso

In [7]:
param_grid = {
    "alpha":[0.0001, 0.001, 0.01, 0.1, 0.2, 0.5, 1],
    "max_iter":[100, 200, 500, 1000]
}

lasso_grid = GridSearchCV(Lasso(), param_grid, n_jobs=-1)
lasso_grid.fit(X_train, y_train)
print(lasso_grid.best_params_)
y_pred = lasso_grid.predict(X_valid)
print("MAE: ", mean_absolute_error(y_valid, y_pred))
print("MSE: ", mean_squared_error(y_valid, y_pred))

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.963e+03, tolerance: 1.892e+00
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.807e+03, tolerance: 1.868e+00
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.070e+03, tolerance: 1.906e

{'alpha': 0.001, 'max_iter': 100}
MAE:  1.1938158833371395
MSE:  2.1868320813644426


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.014e+03, tolerance: 2.356e+00
  model = cd_fast.enet_coordinate_descent(


# LassoCV

In [8]:
param_grid = {
    "n_alphas":[1, 2, 5, 10, 20, 50, 100],
    "max_iter":[20, 50, 100, 200]
}
lassocv_grid = GridSearchCV(LassoCV(), param_grid, n_jobs=-1)
lassocv_grid.fit(X_train, y_train)
print(lassocv_grid.best_params_)
y_pred = lassocv_grid.predict(X_valid)
print("MAE: ", mean_absolute_error(y_valid, y_pred))
print("MSE: ", mean_squared_error(y_valid, y_pred))

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2133.268841164914, tolerance: 1.49648526495308
  model = cd_fast.enet_coordinate_descent_gram(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1980.4601893435356, tolerance: 1.4993031605330267
  model = cd_fast.enet_coordinate_descent_gram(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2473.7395345362565, tolerance: 1.5117857904936165
  model = cd_fast.enet_coordinate_descent_gram(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:617: Convergence

{'max_iter': 20, 'n_alphas': 1}
MAE:  1.2343847994131372
MSE:  2.2725524212217585


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.331e+03, tolerance: 2.356e+00
  model = cd_fast.enet_coordinate_descent(


# Ridge

In [9]:
param_grid = {
    "alpha":[0.0001, 0.001, 0.01, 0.1, 0.2, 0.5, 1],
    "max_iter":[100, 200, 500, 1000]
}

ridge_grid = GridSearchCV(Ridge(), param_grid, n_jobs=-1)
ridge_grid.fit(X_train, y_train)
print(ridge_grid.best_params_)
y_pred = ridge_grid.predict(X_valid)
print("MAE: ", mean_absolute_error(y_valid, y_pred))
print("MSE: ", mean_squared_error(y_valid, y_pred))

{'alpha': 1, 'max_iter': 100}
MAE:  1.1922983245798213
MSE:  2.1858482405790975


# RidgeCV

In [10]:
param_grid = {
    "alphas":[0.0001, 0.001, 0.01, 0.1, 0.2, 0.5, 1],
}

ridgecv_grid = GridSearchCV(RidgeCV(), param_grid, n_jobs=-1)
ridgecv_grid.fit(X_train, y_train)
print(ridgecv_grid.best_params_)
y_pred = ridgecv_grid.predict(X_valid)
print("MAE: ", mean_absolute_error(y_valid, y_pred))
print("MSE: ", mean_squared_error(y_valid, y_pred))

{'alphas': 1}
MAE:  1.1922983249839827
MSE:  2.185848240471387


# ElasticNet

In [11]:
param_grid = {
    "alpha":[0.0001, 0.001, 0.01, 0.1, 0.2, 0.5, 1],
    "max_iter":[100, 200, 500, 1000]
}

en_grid = GridSearchCV(ElasticNet(), param_grid, n_jobs=-1)
en_grid.fit(X_train, y_train)
print(en_grid.best_params_)
y_pred = en_grid.predict(X_valid)
print("MAE: ", mean_absolute_error(y_valid, y_pred))
print("MSE: ", mean_squared_error(y_valid, y_pred))

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.880e+03, tolerance: 1.868e+00
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.033e+03, tolerance: 1.892e+00
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.122e+03, tolerance: 1.906e

{'alpha': 0.0001, 'max_iter': 100}
MAE:  1.191777107243075
MSE:  2.185424450622848


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.743e+03, tolerance: 2.356e+00
  model = cd_fast.enet_coordinate_descent(


# BayesianRidge

In [12]:
param_grid = {
    "n_iter":[100, 200, 500, 1000]
}

br_grid = GridSearchCV(BayesianRidge(), param_grid, n_jobs=-1)
br_grid.fit(X_train, y_train)
print(br_grid.best_params_)
y_pred = br_grid.predict(X_valid)
print("MAE: ", mean_absolute_error(y_valid, y_pred))
print("MSE: ", mean_squared_error(y_valid, y_pred))

{'n_iter': 100}
MAE:  1.192323933078384
MSE:  2.185875672549711


# GradientBoostingRegressor

In [13]:
param_grid = {
    "loss":["squared_error","absolute_error"],
    "n_estimators":[10, 100],
    "max_depth":[3, 5, 10, 20]
}

gbr_grid = GridSearchCV(GradientBoostingRegressor(), param_grid, n_jobs=-1)
gbr_grid.fit(X_train, y_train)
print(gbr_grid.best_params_)
y_pred = gbr_grid.predict(X_valid)
print("MAE: ", mean_absolute_error(y_valid, y_pred))
print("MSE: ", mean_squared_error(y_valid, y_pred))

{'loss': 'squared_error', 'max_depth': 5, 'n_estimators': 100}
MAE:  0.9064140190970715
MSE:  1.5394309434714881


# XGBRegressor

In [14]:
param_grid = {
    "booster":["gbtree","gblinear"],
    "eta":[0.01, 0.02, 0.05, 0.1, 0.2],
    "max_depth":list(range(3,6)),
}

xgbr_grid = GridSearchCV(XGBRegressor(), param_grid, n_jobs=-1)
xgbr_grid.fit(X_train, y_train)
print(xgbr_grid.best_params_)
y_pred = xgbr_grid.predict(X_valid)
print("MAE: ", mean_absolute_error(y_valid, y_pred))
print("MSE: ", mean_squared_error(y_valid, y_pred))

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [08:19:03] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [08:19:03] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [08:19:03] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [08:19:03] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [08:19:03] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_depth" } are not used.

  warnings.warn

{'booster': 'gbtree', 'eta': 0.1, 'max_depth': 4}
MAE:  0.915958986559473
MSE:  1.5517728873253998


# NN Regression

In [15]:
normalizer = tf.keras.layers.Normalization(input_shape = (11,), axis=None)
normalizer.adapt(X_train)
neuralnet = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(1)
])

In [16]:
neuralnet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), loss="mean_squared_error")
neuralnet.fit(X_train, y_train, verbose=0, epochs=1000, validation_data=(X_valid,y_valid))

In [17]:
y_pred = neuralnet.predict(X_valid)
print("MAE: ", mean_absolute_error(y_valid, y_pred))
print("MSE: ", mean_squared_error(y_valid, y_pred))

66/66 [==============================] - 0s 1ms/step
MAE:  1.2245444437955915
MSE:  2.2676598732311666


# Choosing Model

In [18]:
model = VotingRegressor([
    ("1", linreg),
    ("2", lasso_grid),
    ("3", lassocv_grid),
    ("4", ridge_grid),
    ("5", ridgecv_grid),
    ("6", en_grid),
    ("7", br_grid),
    ("8", gbr_grid),
    ("9", xgbr_grid),
])

In [19]:
model.fit(X_train, y_train)
y_pred1 = model.predict(X_valid)
y_pred2 = neuralnet.predict(X_valid)
print("Regressor Model")
print("MAE: ", mean_absolute_error(y_valid, y_pred1))
print("MSE: ", mean_squared_error(y_valid, y_pred1))
print("Tensorflow Neuralnet Model")
print("MAE: ", mean_absolute_error(y_valid, y_pred2))
print("MSE: ", mean_squared_error(y_valid, y_pred2))

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.807e+03, tolerance: 1.868e+00
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.963e+03, tolerance: 1.892e+00
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.953e+03, tolerance: 1.886e

66/66 [==============================] - 0s 1ms/step
Regressor Model
MAE:  1.1109813971006177
MSE:  1.9334074533037973
Tensorflow Neuralnet Model
MAE:  1.2245444437955915
MSE:  2.2676598732311666


# Predicting Test Data

In [20]:
if (mean_absolute_error(y_valid, y_pred1) < mean_absolute_error(y_valid, y_pred2)) and (mean_squared_error(y_valid, y_pred1) < mean_squared_error(y_valid, y_pred2)):
    final_model = model
elif (mean_absolute_error(y_valid, y_pred1) > mean_absolute_error(y_valid, y_pred2)) and (mean_squared_error(y_valid, y_pred1) > mean_squared_error(y_valid, y_pred2)):
    final_model = neuralnet
else:
    if mean_absolute_error(y_valid, y_pred1) + mean_squared_error(y_valid, y_pred1) < mean_absolute_error(y_valid, y_pred2) + mean_squared_error(y_valid, y_pred2):
        final_model = model
    else:
        final_model = neuralnet

In [21]:
pred_id = raw_test_data["id"]
X_test = raw_test_data.drop(columns=["id"])
y_pred = final_model.predict(X_test)
final = pd.DataFrame(y_pred, index=pred_id)
final.columns=["Hardness"]
print(final)

       Hardness
id             
10407  3.331833
10408  4.396656
10409  5.337579
10410  5.073346
10411  5.058596
...         ...
17341  5.576582
17342  4.346456
17343  5.406894
17344  4.447462
17345  1.869760

[6939 rows x 1 columns]


In [22]:
final.to_csv("submission.csv")